# AG2: Orchestrating agents (Sequential chat)

- Orchestrating agents<br>
  - https://docs.ag2.ai/docs/user-guide/basic-concepts/orchestration/orchestrations
- Sequential chat<br>
  - https://docs.ag2.ai/docs/user-guide/basic-concepts/orchestration/sequential-chat

## Orchestrating agents

- **Two-agent chat**: The simplest form of conversation pattern where two agents chat back-and-forth with each other. This has been demonstrated in the previous examples.
- **Sequential chat**: A sequence of chats, each between two agents, chained together by a carryover mechanism (which brings the summary of the previous chat to the context of the next chat). Useful for simple sequential workflows.
- **Group chat**: A chat with more than two agents with options on how agents are selected. See GroupChat Overview for further details.
- **Nested chat**: A mechanism to package a workflow into a single agent/chat for reuse in a workflow.
- **Swarm**: A pattern based on agents with handoffs. There’s a shared context and each agent has tools and the ability to transfer control to other agents. The original swarm concept was created by OpenAI.

## SETUP

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables (for API key)
load_dotenv()

# Set up OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Please set the OPENAI_API_KEY environment variable or add it to a .env file")

# Define the model to use
MODEL_GPT = "gpt-4o-mini"

## Sequential chat

In [2]:
from autogen import ConversableAgent

llm_config = {"api_type": "openai", "model": "gpt-4o-mini"}

# Three chats:
# 1. Teacher and Curriculum designer > summary is a topic for next chat
# 2. Teacher and Lesson planner (with 1 revision) > summary is lesson plan for next chat
# 3. Teacher and Formatter > summary is a formatted lesson plan

# Curriculum designer
curriculum_message = """You are a curriculum designer for a fourth grade class. Nominate an appropriate a topic for a lesson, based on the given subject."""

lesson_curriculum = ConversableAgent(
    name="curriculm_agent",
    llm_config=llm_config,
    system_message=curriculum_message,
)

# Lesson planner
planner_message = """You are a classroom lesson agent.
Given a topic, write a lesson plan for a fourth grade class in bullet points. Include the title, learning objectives, and script.
"""

lesson_planner = ConversableAgent(
    name="planner_agent",
    llm_config=llm_config,
    system_message=planner_message,
)

# Formatter
formatter_message = """You are a lesson plan formatter. Format the complete plan as follows:
<title>Lesson plan title</title>
<learning_objectives>Key learning objectives</learning_objectives>
<script>How to introduce the topic to the kids</script>
"""

lesson_formatter = ConversableAgent(
    name="formatter_agent",
    llm_config=llm_config,
    system_message=formatter_message,
)

# Teacher who initiates the chats
teacher_message = """You are a classroom teacher.
You decide topics for lessons and work with a lesson planner, you provide one round of feedback on their lesson plan.
Then you will work with a formatter to get a final output of the lesson plan.
"""

teacher = ConversableAgent(
    name="teacher_agent",
    llm_config=llm_config,
    system_message=teacher_message,
)

In [3]:
# Our sequential chat, each chat is a chat between the teacher and the recipient agent
# max_turns determines if there's back and forth between the teacher and the recipient
# max_turns = 1 means no back and forth
chat_results = teacher.initiate_chats(
    [
        {
            "recipient": lesson_curriculum,
            "message": "Let's create a science lesson, what's a good topic?",
            "max_turns": 1,
            "summary_method": "last_msg",
        },
        {
            "recipient": lesson_planner,
            "message": "Create a lesson plan.",
            "max_turns": 2, # One revision
            "summary_method": "last_msg",
        },
        {
            "recipient": lesson_formatter,
            "message": "Format the lesson plan.",
            "max_turns": 1,
            "summary_method": "last_msg",
        },
    ]
)


********************************************************************************
Starting a new chat....

********************************************************************************
teacher_agent (to curriculm_agent):

Let's create a science lesson, what's a good topic?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
curriculm_agent (to teacher_agent):

A great topic for a fourth-grade science lesson is "Ecosystems and Habitats." This topic allows students to explore the relationships between plants, animals, and their environments. It offers opportunities for hands-on activities, such as creating a model ecosystem or conducting a simple investigation about local habitats. Students can learn about different ecosystems (like forests, deserts, oceans, etc.), the organisms that inhabit them, and the importance of biodiversity and conservation. This topic is engaging and aligns well with the fourth-grade science standard

In [4]:
# The result of `initiate_chats` is a list of chat results
# each chat result has a summary
print("\n\nCurriculum summary:\n", chat_results[0].summary)
print("\n\nLesson Planner summary:\n", chat_results[1].summary)
print("\n\nFormatter summary:\n", chat_results[2].summary)



Curriculum summary:
 A great topic for a fourth-grade science lesson is "Ecosystems and Habitats." This topic allows students to explore the relationships between plants, animals, and their environments. It offers opportunities for hands-on activities, such as creating a model ecosystem or conducting a simple investigation about local habitats. Students can learn about different ecosystems (like forests, deserts, oceans, etc.), the organisms that inhabit them, and the importance of biodiversity and conservation. This topic is engaging and aligns well with the fourth-grade science standards!


Lesson Planner summary:
 Thank you for your thoughtful feedback! I appreciate your suggestions for enhancing the lesson plan. Below is the revised version that incorporates your recommendations:

---

### Enhanced Lesson Plan: Ecosystems and Habitats

**Grade Level:** 4th Grade  
**Subject:** Science  
**Duration:** 60 minutes  

---

**Learning Objectives:**
- Students will define ecosystems an